In [1]:
import pandas as pd
import os , sys, time
import numpy as np

import hail as hl
from bokeh.io import output_notebook, show, export_png
import bokeh.models.layouts as bl

/home/barbitoff/.conda/envs/hail/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
os.environ['JAVA_HOME']='/usr/lib/jvm/java-1.8.0-openjdk-amd64'
os.environ['PATH'] = os.environ['PATH'] + f':{os.environ["JAVA_HOME"]}/bin'

In [3]:
hl.init()
output_notebook()

Running on Apache Spark version 2.4.1
SparkUI available at http://217.148.215.30:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.63-cb767a7507c8
LOGGING: writing to /media/array/guar_proj/production_callset/hail-20210428-1540-0.2.63-cb767a7507c8.log


Loading BokehJS ...

In [4]:
ref_g = hl.genetics.ReferenceGenome.from_fasta_file("GUAR_masurca", 
                   "/media/array/guar_proj/reference_masurca/assembly_guar_Masurca1.fasta", 
                   "/media/array/guar_proj/reference_masurca/assembly_guar_Masurca1.fasta.fai")

In [5]:
mt = hl.import_vcf('/media/array/guar_proj/production_callset/GATK_NGSEP_TASSEL_common.vcf', reference_genome=ref_g)
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GUAR_masurca>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        BaseQRankSum: float64, 
        DP: int32, 
        DS: bool, 
        END: int32, 
        ExcessHet: float64, 
        FS: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQRankSum: float64, 
        QD: float64, 
        RAW_MQandDP: array<int32>, 
        ReadPosRankSum: float64, 
        SOR: float64, 
        NGSEP_NHET: array<int32>, 
        NGSEP_NALT: array<int32>, 
        NGSEP_AF: array<float64>, 
        TASSEL_NHET: array<int32>, 
        TASSEL_NALT: array<int32>, 
        TASSEL

In [6]:
mt.count()

2021-04-28 15:40:44 Hail: INFO: Coerced sorted dataset


(21311, 192)

In [7]:
mt = hl.sample_qc(mt)
mt = hl.variant_qc(mt)

In [8]:
mt.variant_qc.describe()

--------------------------------------------------------
Type:
        struct {
        dp_stats: struct {
            mean: float64, 
            stdev: float64, 
            min: float64, 
            max: float64
        }, 
        gq_stats: struct {
            mean: float64, 
            stdev: float64, 
            min: float64, 
            max: float64
        }, 
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        homozygote_count: array<int32>, 
        call_rate: float64, 
        n_called: int64, 
        n_not_called: int64, 
        n_filtered: int64, 
        n_het: int64, 
        n_non_ref: int64, 
        het_freq_hwe: float64, 
        p_value_hwe: float64
    }
--------------------------------------------------------
Source:
Index:
    ['row']
--------------------------------------------------------


In [9]:
p = hl.plot.histogram(mt.variant_qc.call_rate)
show(p)

2021-04-28 15:40:49 Hail: INFO: Coerced sorted dataset
2021-04-28 15:40:56 Hail: INFO: Coerced sorted dataset


In [10]:
p = hl.plot.histogram(mt.variant_qc.n_het)
show(p)

2021-04-28 15:41:04 Hail: INFO: Coerced sorted dataset
2021-04-28 15:41:10 Hail: INFO: Coerced sorted dataset


In [11]:
p = hl.plot.histogram(hl.min(mt.info.AF))
show(p)

2021-04-28 15:41:19 Hail: INFO: Coerced sorted dataset
2021-04-28 15:41:23 Hail: INFO: Coerced sorted dataset


In [12]:
mt_flt = mt.filter_rows(mt.variant_qc.n_het < 96)
mt_flt = mt_flt.filter_rows(mt_flt.variant_qc.call_rate > 0.75)
#mt_flt = mt_flt.filter_rows(hl.min(mt_flt.variant_qc.AF) > 0.05)

In [13]:
p = hl.plot.histogram(mt_flt.variant_qc.gq_stats.mean)
show(p)

2021-04-28 15:41:27 Hail: INFO: Coerced sorted dataset
2021-04-28 15:41:32 Hail: INFO: Coerced sorted dataset


In [14]:
mt_flt = mt_flt.filter_rows(mt_flt.variant_qc.gq_stats.mean > 15)

In [15]:
mt_flt.count()

2021-04-28 15:41:38 Hail: INFO: Coerced sorted dataset


(13108, 192)

In [16]:
p = hl.plot.histogram(mt_flt.variant_qc.n_het)
show(p)

2021-04-28 15:41:43 Hail: INFO: Coerced sorted dataset
2021-04-28 15:41:48 Hail: INFO: Coerced sorted dataset


In [17]:
p = hl.plot.histogram(hl.min(mt_flt.info.AF))
show(p)

2021-04-28 15:41:54 Hail: INFO: Coerced sorted dataset
2021-04-28 15:42:00 Hail: INFO: Coerced sorted dataset


In [18]:
p = hl.plot.histogram(mt_flt.sample_qc.r_het_hom_var)
show(p)

2021-04-28 15:42:06 Hail: INFO: Coerced sorted dataset
2021-04-28 15:42:11 Hail: INFO: Coerced sorted dataset
2021-04-28 15:42:13 Hail: INFO: Coerced sorted dataset
2021-04-28 15:42:18 Hail: INFO: Coerced sorted dataset


In [19]:
p = hl.plot.histogram(mt_flt.sample_qc.call_rate)
show(p)

2021-04-28 15:42:21 Hail: INFO: Coerced sorted dataset
2021-04-28 15:42:26 Hail: INFO: Coerced sorted dataset
2021-04-28 15:42:28 Hail: INFO: Coerced sorted dataset
2021-04-28 15:42:33 Hail: INFO: Coerced sorted dataset


In [20]:
mt_flt = mt_flt.filter_cols(mt_flt.sample_qc.call_rate > 0.8)
mt_flt = mt_flt.filter_cols(mt_flt.sample_qc.r_het_hom_var < 2)
mt_flt.count()

2021-04-28 15:42:36 Hail: INFO: Coerced sorted dataset
2021-04-28 15:42:41 Hail: INFO: Coerced sorted dataset


(13108, 166)

In [21]:
mt_flt = hl.variant_qc(mt_flt)

In [22]:
mt_flt = mt_flt.filter_rows(hl.min(mt_flt.variant_qc.AF) > 0)

In [23]:
mt_flt.count()

2021-04-28 15:42:48 Hail: INFO: Coerced sorted dataset
2021-04-28 15:42:53 Hail: INFO: Coerced sorted dataset


(12602, 166)

In [29]:
hl.export_vcf(mt_flt, '/media/array/guar_proj/production_callset/GUAR_2of3_noAF_noQD_filter.vcf')

2021-04-28 16:26:16 Hail: WARN: export_vcf: ignored the following fields:
    'sample_qc' (column)
    'variant_qc' (row)
2021-04-28 16:26:18 Hail: INFO: Coerced sorted dataset
2021-04-28 16:26:20 Hail: INFO: Coerced sorted dataset
2021-04-28 16:26:28 Hail: INFO: merging 1 files totalling 67.3M...
2021-04-28 16:26:29 Hail: INFO: while writing:
    /media/array/guar_proj/production_callset/GUAR_2of3_noAF_noQD_filter.vcf
  merge time: 174.440ms


In [24]:
mt_common = mt_flt.filter_rows(hl.min(mt_flt.variant_qc.AF) > 0.05)
mt_common.count()

2021-04-28 15:43:02 Hail: INFO: Coerced sorted dataset
2021-04-28 15:43:07 Hail: INFO: Coerced sorted dataset


(5760, 166)

In [25]:
p = hl.plot.histogram(mt_common.variant_qc.gq_stats.mean)
show(p)

2021-04-28 15:43:14 Hail: INFO: Coerced sorted dataset
2021-04-28 15:43:18 Hail: INFO: Coerced sorted dataset
2021-04-28 15:43:24 Hail: INFO: Coerced sorted dataset
2021-04-28 15:43:29 Hail: INFO: Coerced sorted dataset


In [26]:
p = hl.plot.histogram(mt_common.info.QD)
show(p)

2021-04-28 15:43:46 Hail: INFO: Coerced sorted dataset
2021-04-28 15:43:51 Hail: INFO: Coerced sorted dataset
2021-04-28 15:44:01 Hail: INFO: Coerced sorted dataset
2021-04-28 15:44:06 Hail: INFO: Coerced sorted dataset


In [27]:
mt_common = mt_common.filter_rows(mt_common.info.QD > 20)
mt_common.count()

2021-04-28 15:45:24 Hail: INFO: Coerced sorted dataset
2021-04-28 15:45:31 Hail: INFO: Coerced sorted dataset


(5234, 166)

In [28]:
hl.export_vcf(mt_common, '/media/array/guar_proj/production_callset/GUAR_2of3_AF5pct.vcf')

2021-04-28 15:46:00 Hail: WARN: export_vcf: ignored the following fields:
    'sample_qc' (column)
    'variant_qc' (row)
2021-04-28 15:46:03 Hail: INFO: Coerced sorted dataset
2021-04-28 15:46:08 Hail: INFO: Coerced sorted dataset
2021-04-28 15:46:28 Hail: INFO: merging 1 files totalling 27.5M...
2021-04-28 15:46:29 Hail: INFO: while writing:
    /media/array/guar_proj/production_callset/GUAR_2of3_AF5pct.vcf
  merge time: 378.127ms


In [38]:
pd_df = mt_common.rows().to_pandas()

2021-04-13 18:39:35 Hail: INFO: Coerced sorted dataset
2021-04-13 18:39:36 Hail: INFO: Coerced sorted dataset


In [40]:
pd_df.to_csv('GUAR_2of3_AF5pct_quality.tsv', sep='\t')